In [91]:
import os
import struct
import numpy as np
import math
import matplotlib.pyplot as plt
import copy
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from numpy import linalg as LA
cwd = os.getcwd()  # Get the current working directory (cwd)
x_train_p = os.path.join(cwd, 'train-images.idx3-ubyte')
y_train_p = os.path.join(cwd, 'train-labels.idx1-ubyte')
x_test_p = os.path.join(cwd, 't10k-images.idx3-ubyte')
y_test_p = os.path.join(cwd, 't10k-labels.idx1-ubyte')

# Load everything in numpy arrays
with open(y_train_p, 'rb') as y_train_f:
    magic, num = struct.unpack(">II", y_train_f.read(8))
    y_train = np.fromfile(y_train_f, dtype=np.int8)
with open(y_test_p, 'rb') as y_test_f:
    magic, num = struct.unpack(">II", y_test_f.read(8))
    y_test = np.fromfile(y_test_f, dtype=np.int8)
with open(x_train_p, 'rb') as x_train_f:
    magic, num, rows, cols = struct.unpack(">IIII", x_train_f.read(16))
    x_train = np.fromfile(x_train_f, dtype=np.uint8).reshape(len(y_train), rows*cols)
with open(x_test_p, 'rb') as x_test_f:
    magic, num, rows, cols = struct.unpack(">IIII", x_test_f.read(16))
    x_test = np.fromfile(x_test_f, dtype=np.uint8).reshape(len(y_test), rows*cols)
im_size = x_train.shape[1]
train_size = x_train.shape[0]
test_size = x_test.shape[0]





train_mean=np.mean(x_train,axis=0)
train_mean[0]=0
train_std=np.std(x_train,axis=0)
for i in range(im_size):
    if train_std[i]==0.:
        train_std[i]=1
x_train=(x_train-train_mean)/train_std
x_test=(x_test-train_mean)/train_std


t_train=np.zeros((train_size,10))
t_test=np.zeros((test_size,10))
for i in range(train_size):
    t_train[i][y_train[i]]=1
for i in range(test_size):
    t_test[i][y_test[i]]=1
    
iteration=5000
input_layer=28*28
hidden_layer=10
output_layer=10

eta=1e-5
_lambda=0
batch_size=60000
num_batch=1
wt_lim=0.01

In [94]:
# weights and gradients initialization
#W1=wt_lim*np.random.randn((input_layer+1,hidden_layer))
#W2=wt_lim*np.random.randn((hidden_layer+1,output_layer))
W1 = wt_lim * np.random.randn(input_layer+1, hidden_layer)
W2 = wt_lim * np.random.randn(hidden_layer+1, output_layer)
G1=np.zeros((input_layer+1,hidden_layer))
G2=np.zeros((hidden_layer+1,output_layer))

In [ ]:
from numba import jit
@jit
def shuffle(x,y):
    np.random.seed(1234)
    p=np.random.permutation(len(x))
    return x[p],y[p]
x_train,y_train=shuffle(x_train,ytrain)    
@jit
def sigmoid(x):
    return 1./(1+np.exp(-x))
@jit 
def softmax(x):
    mx=np.amax(x,axis=0)
    sm=np.sum(np.exp(x-mx),axis=0)
    return np.exp(x-mx)/sm
@jit 
def sigmoid_prime(x):
    return x*(1-x)

@jit
def error(output_layer_state, Y):
    mx = np.argmax(output_layer_state, axis=0).reshape(-1, 1)
    return (100 * ((mx!=Y.reshape(-1, 1))*1.0).sum()/len(Y))

@jit
def forward_prop(X):
    # Forward propagation
    a, b = X.shape
    X = np.vstack((np.ones((1,b)), X)) # Add 1 for bias term
    hidden_layer_state = sigmoid(W1.T.dot(X))
    hidden_layer_state = np.vstack((np.ones((1,b)), hidden_layer_state)) # Add 1 for bias term
    output_layer_state = softmax(W2.T.dot(hidden_layer_state))
    return hidden_layer_state, output_layer_state

In [ ]:
error_train_list=list()
error_test_list=list()
for epoch in range(iteration):
    
    
    
    (hidden_layer_state, output_layer_state) = forward_prop(x_train.T)
    # Back propagation
    d2 = t_train.T - output_layer_state
    G2 = -hidden_layer_state.dot(d2.T)
    d1 =  W2.dot(d2)[1:] * sigmoid_prime(hidden_layer_state[1:])

    G1 = -np.vstack((np.ones((1,batch_size)), x_train.T)).dot(d1.T)

    G1 = G1 + _lambda * np.vstack((np.zeros((1,hidden_layer)), ((W1[1:,:]))))
 
    G2 = G2 + _lambda * np.vstack((np.zeros((1,output_layer)), W2[1:,:]))

    W1=W1-eta*G1
    W2=W2-eta*G2
    # calculuate error of training set after each epoch

    #(hidden_layer_state, output_layer_state) = fprop(X_train_split.T)
    acctrain = error(output_layer_state, y_train)
    error_train_list.append(acctrain)
    (hidden_layer_state, output_layer_state) = forward_prop(x_test.T)
    test_err = error(output_layer_state, y_test)
    error_test_list.append(test_err)
    print('After ', (epoch+1), 'th epoch, training error', train_err)
    print('After ', (epoch+1), 'th epoch, test error', test_err)

In [ ]:
plt.plot(range(1,len(error_train_list)+1),error_train_list,label='Train')
plt.plot(range(1,len(error_test_list)+1),error_test_list,label='Test')
plt.xlabel('Iterations')
plt.ylabel('Probability of Error(%)')
legend=plt.legend()
for label in legend.get_texts():
    label.set_fontsize('large')
for label in legend.get_lines():
    label.set_linewidth('1.5')